# Scoring of MCQ and FCQ from Amy's data
We will read in raw response data, combine with the WCQ and FCQ questions, and score each. This will result in a $\log(k)$ score for each participant for the WCQ and the FCQ.

WCQ = Lim & Bruce.

FCQ = Hendrickson et al (2015)

Information about use of `pm.Data` containers can be found here https://docs.pymc.io/notebooks/data_container.html. 

In [ ]:
# Install Black autoformatter with: pip install nb-black
%load_ext lab_black

import pymc3 as pm

import numpy as np
import pandas as pd

# plotting
%config InlineBackend.figure_format = 'retina'
import matplotlib.pyplot as plt

plt.rcParams.update({"font.size": 14})

# Define options

In [ ]:
SEED = 12345

# Define sampler options
sample_options = {
    "tune": 2000,
    "draws": 2000,
    "chains": 2,
    "cores": 2,
    "nuts_kwargs": {"target_accept": 0.95},
    "random_seed": SEED,
}

# Define plotting functions

In [ ]:
def plot_discount_func(ax, data, trace):
    delays = np.linspace(0, np.max(data.DB.values), 1000)

    # plot posterior mean
    k = np.exp(np.mean(trace["logk"]))
    ax.plot(delays, discount_function(delays, k), lw=4)

    # plot 95% region
    p = np.percentile(np.exp(trace["logk"]), [5 / 2, 100 - (5 / 2)])
    ax.fill_between(
        delays,
        discount_function(delays, p[0]),
        discount_function(delays, p[1]),
        alpha=0.2,
    )

In [ ]:
def plot_data(data, ax=None):

    if ax is None:
        ax = plt.gca()

    D = data["R"] == 1
    I = data["R"] == 0

    if np.sum(D) > 0:
        ax.scatter(
            x=data["DB"][D],
            y=data["RA"][D] / data["RB"][D],
            c="k",
            edgecolors="k",
            label="chose delayed prospect",
        )
    if np.sum(I) > 0:
        ax.scatter(
            x=data["DB"][I],
            y=data["RA"][I] / data["RB"][I],
            c="w",
            edgecolors="k",
            label="chose immediate prospect",
        )

    ax.set(
        xlabel="DB", ylabel="RA/RB", ylim=[0, 1], xlim=[0, 1.05 * np.max(data["DB"])]
    )

In [ ]:
def plot_questions(data, ax=None):

    if ax is None:
        ax = plt.gca()

    ax.scatter(x=data["DB"], y=data["RA"] / data["RB"])
    ax.set(
        xlabel="DB", ylabel="RA/RB", ylim=[0, 1], xlim=[0, 1.05 * np.max(data["DB"])]
    )

# Load raw data

In [ ]:
data = pd.read_csv("../02 processed data/study2_processed.csv")

# Read in question values from WCQ and FCQ
We need the delay and reward values for the WCQ and the FCQ. We will read these in from `.csv` files.

In [ ]:
wcq = pd.read_csv("study2_wcq.csv")

# IMPORTANT: Ensure rows are sorted by `order`
wcq = wcq.sort_values(by="order")

plot_questions(wcq)

In [ ]:
wcq

In [ ]:
fcq = pd.read_csv("study2_fcq.csv")

# IMPORTANT: Ensure rows are sorted by `order`
fcq = fcq.sort_values(by="order")

plot_questions(fcq)

## Data extraction functions
These functions will get the responses from the raw data file, and combine them together with the MCQ or WCQ questions we imported.

In [ ]:
def extract_WCQ_data(data, row):
    id = data.iloc[row, :].URN
    df = wcq
    df["R"] = (data.iloc[row, data.columns.str.contains("WCQ")] - 1).values
    # force to be numeric
    df = df.astype(float)
    return (id, df)

In [ ]:
def extract_FCQ_data(data, row):
    id = data.iloc[row, :].URN
    df = fcq
    df["R"] = (data.iloc[row, data.columns.str.contains("FCQ")] - 1).values
    # force to be numeric
    df = df.astype(float)
    return (id, df)

# Build our Bayesian model
We will use the `pm.Data` class so that we can build one model only, then use it multiple times to fit data from each participant seperately. This should make things more efficient, avoiding building the same model hundreds of times.

In [ ]:
def V(reward, delay, logk):
    """Calculate the present subjective value of a given prospect"""
    k = pm.math.exp(logk)
    return reward * discount_function(delay, k)


def discount_function(delay, k):
    """ Hyperbolic discount function """
    return 1 / (1.0 + (k * delay))


def Φ(VA, VB, ϵ=0.01):
    """Psychometric function which converts the decision variable (VB-VA)
    into a reponse probability. Output corresponds to probability of choosing
    the delayed reward (option B)."""
    return ϵ + (1.0 - 2.0 * ϵ) * (1 / (1 + pm.math.exp(-1.7 * (VB - VA))))

In [ ]:
def build_model(data):
    with pm.Model() as model:

        # data nodes
        RA = pm.Data("RA", data.RA.values)
        RB = pm.Data("RB", data.RB.values)
        DB = pm.Data("DB", data.DB.values)
        R = pm.Data("R", data.R.values)

        # prior
        logk = pm.Normal("logk", mu=-3, sd=2)

        # response probability
        P = pm.Deterministic("P", Φ(RA, V(RB, DB, logk)))

        # likelihood
        response = pm.Bernoulli("response", p=P, observed=R)

    return model

In [ ]:
def score_participant(data, plot=False):
    """Our core function to score a participant"""

    with model:
        # set the data
        pm.set_data({"RA": data.RA, "RB": data.RB, "DB": data.DB, "R": data.R})

        # do the sampling
        trace = pm.sample(**sample_options)

    logk_mean = np.mean(trace["logk"])

    if plot:
        fig, ax = plt.subplots(figsize=(6, 4))
        plot_data(data, ax=ax)
        plot_discount_func(ax, data, trace)
        plt.show()

    return logk_mean

# Score FCQ

In [ ]:
# in order to build the model we need some example data
participant = 0
temp_trial_data = extract_FCQ_data(data, participant)[1]
temp_trial_data

# Now build the model at last
model = build_model(temp_trial_data)

In [ ]:
pm.model_to_graphviz(model)

In [ ]:
n_participants = data.shape[0]

should_plot = False

pid = []
logk_fcq = []

for i in range(n_participants):
    id_num, fcq_trial_data = extract_FCQ_data(data, i)
    logk_fcq_value = score_participant(fcq_trial_data, plot=should_plot)
    logk_fcq.append(logk_fcq_value)
    pid.append(id_num)

results = pd.DataFrame({"URN": pid, "logk_fcq": logk_fcq})

In [ ]:
# merge with original data file
data = pd.merge(data, results, on="URN")

# Score WCQ

In [ ]:
# in order to build the model we need some example data
participant = 0
temp_trial_data = extract_WCQ_data(data, participant)[1]
temp_trial_data

# Now build the model at last
model = build_model(temp_trial_data)

In [ ]:
pm.model_to_graphviz(model)

In [ ]:
n_participants = data.shape[0]

should_plot = False

pid = []
logk_wcq = []

for i in range(n_participants):
    id_num, wcq_trial_data = extract_WCQ_data(data, i)
    print(id_num)
    logk_wcq_value = score_participant(wcq_trial_data, plot=should_plot)
    logk_wcq.append(logk_wcq_value)
    pid.append(id_num)

results = pd.DataFrame({"URN": pid, "logk_wcq": logk_wcq})

In [ ]:
# merge with original data file
data = pd.merge(data, results, on="URN")

## Wrap up

In [ ]:
# remove raw data columns, no longer needed
data = data[data.columns.drop(list(data.filter(regex="WCQ")))]
data = data[data.columns.drop(list(data.filter(regex="FCQ")))]

In [ ]:
# export
data.to_csv("study2_final_data.csv", index=False)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(12, 4), sharex=False)

bins = 21

ax[0].hist(logk_wcq, bins)
ax[0].set(
    xlabel="$\ln(k)$ [k in units of days$^{-1}$]", ylabel="frequency", title="WCQ"
)

ax[1].hist(logk_fcq, bins)
ax[1].set(
    xlabel="$\ln(k)$ [k in units of hours$^{-1}$]", ylabel="frequency", title="FCQ"
)

# increased space between rows
plt.subplots_adjust(hspace=0.4)

plt.savefig("../study2_fit_histograms.pdf", bbox_inches="tight", dpi=300)